# Compute i coefficients

In [ ]:
import brightway2 as bw
import pandas as pd
import os
import numpy as np

# Set working directry
path = "."
os.chdir(path)

# Local functions
from utils.lookup_func import lookup_geothermal

# Set project
bw.projects.set_current("Geothermal")

# Retrieve activities
wellhead, diesel, steel, cement, water, \
drilling_mud, drill_wst, wells_closr, coll_pipe, \
plant, ORC_fluid, ORC_fluid_wst, diesel_stim, co2,_, _ = lookup_geothermal()
cooling_tower=bw.Database("geothermal energy").search("cooling tower")[0].key

list_act = [wellhead, diesel, steel, cement, water, 
         drilling_mud, drill_wst, wells_closr, coll_pipe,
         plant, cooling_tower, ORC_fluid, ORC_fluid_wst, diesel_stim]

# Retrieve methods 
ILCD_CC = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "climate change total" in str(method)]
ILCD_HH = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "human health" in str(method)]
ILCD_EQ = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "ecosystem quality" in str(method)]
ILCD_RE = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "resources" in str(method)]
ILCD = ILCD_CC + ILCD_HH + ILCD_EQ + ILCD_RE

# Calculate impact of activities
lca = bw.LCA({list_act[0]: 1}, ILCD[0])
lca.lci()
lca.lcia()
coeff = {}
for method in ILCD:
    s=[]
    lca.switch_method(method)
    for act in list_act:
        lca.redo_lcia({act: 1})
        s.append(lca.score)
    coeff[method] = s

# Retrieve CF for co2 emissions    
for method in ILCD:
    CFs = bw.Method(method).load()
    coeff[method].append(next((cf[1] for cf in CFs if cf[0]==co2),0))

# Build matrix
col_names = ["wellhead", "diesel", "steel", "cement", "water", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid", "ORC_fluid_wst", "diesel_stim", "co2"]
coeff_matrix = pd.DataFrame.from_dict(coeff, orient="index", columns=col_names)

#Constant
drilling_waste_per_metre = 450

# Re-arrange matrix
coeff_matrix["concrete"] = coeff_matrix["cement"] + coeff_matrix["water"] * 1/0.65
coeff_matrix["ORC_fluid_tot"] = coeff_matrix["ORC_fluid"] - coeff_matrix["ORC_fluid_wst"]
coeff_matrix["electricity_stim"] = coeff_matrix["diesel_stim"] * 3.6 / 0.3
coeff_matrix["drill_wst"] = coeff_matrix["drill_wst"] * -1 * drilling_waste_per_metre
coeff_matrix=coeff_matrix.drop(columns=["cement", "ORC_fluid", "ORC_fluid_wst", "diesel_stim"])

col_ord = ["wellhead", "diesel", "steel", "concrete", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid_tot", "water", "electricity_stim", "co2"]
coeff_matrix=coeff_matrix[col_ord]

is_= ["i1", "i2.1", "i2.2", "i2.3", "i2.4", "i2.5", \
      "i2.6","i3", "i4.1", "i4.2", "i4.3", "i5.1", "i5.2", "i6"]

if len(coeff_matrix.columns) == len(is_):
    coeff_matrix.columns=is_

In [ ]:
coeff_matrix


# Enhanced geothermal

# Define symbolic

In [ ]:
from sympy import symbols, collect, ratsimp, fraction, factor_terms

In [ ]:
# Coefficients
i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6 = \
symbols('i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6')

# Main parameters
W_Pn, P_ne , PIratio, W_d, E_co2, SR_p, D_i, LT, SR_m, SR_e, D, C_S, C_C, DM, CP, CF, AP, SW_n, S_w, S_el  = \
symbols('W_Pn, P_ne, PIratio, W_d, E_co2, SR_p, D_i, LT, SR_m, SR_e, D, C_S, C_C, DM, CP, CF, AP, SW_n, SW, S_el')

# Other variables that are actually irrelevant, because they will be multiplied by 0 in the enhanced model
# I wanna keep them though for the sake of completeness
E_co2 = symbols('E_co2')

# Constants
CT_el, CT_n, W_en, OF, DW = symbols ('CT_el, CT_n, W_E_en, OF, DW')


# Main equation


In [ ]:
# Number of wells with success rate
W_n = W_Pn/SR_p
W_E_en = W_en/SR_e

In [ ]:
# Equation
nominator = (W_n + W_E_en) *i1 \
          + W_d* (W_n + W_E_en) * (D*i2_1 + C_S*i2_2 + C_C*i2_3  +  DM*i2_4 + DW*i2_5 + i2_6) + \
          + W_n*CP*i3 \
          + P_ne*i4 \
          + W_n*SW_n*S_w* (i5_1 + S_el*i5_2)

c1 = P_ne*CF*(1-AP)*LT*8760000
d1 = CT_el*CT_n*1000*LT
denominator = c1  -  d1

summand = E_co2*i6

eq = nominator/denominator + summand
eq = eq.subs(i4, i4_1 + i4_2*CT_n + i4_3*OF)

In [ ]:
eq

# Get parameters value

In [ ]:
# Klausen
from ege_klausen import parameters
parameters.static()   

# Parameters of simplified models
D_val      = parameters["specific_diesel_consumption"]
W_d_val      = parameters["average_depth_of_wells"]
P_ne_val     = parameters["installed_capacity"]
SR_p_val     = parameters["success_rate_primary_wells"]/100

# Parameters to be fixed
LT_val     = parameters["lifetime"]
SR_e_val   = parameters["success_rate_exploration_wells"] / 100
C_S_val    = parameters["specific_steel_consumption"]
C_C_val    = parameters["specific_cement_consumption"]
DM_val     = parameters["specific_drilling_mud_consumption"]
CP_val     = parameters["collection_pipelines"]
CF_val     = parameters["capacity_factor"]
AP_val     = parameters["auxiliary_power"]
W_Pn_val   = parameters["number_of_wells"] 
SW_n_val   = np.round(0.5 + parameters["number_of_wells_stimulated_0to1"] * parameters["number_of_wells"])
S_w_val    = parameters["water_stimulation"]
S_el_val   = parameters["specific_electricity_stimulation"] / 1000

# Parameters that are not used in ENHANCED
D_i_val     = 0
E_co2_val   = 0

# Constants
CT_el_val  = 864
CT_n_val   = 7/303.3
W_en_val   = 3 * 0.3
OF_val     = 300
DW_val     = 450

par_dict = { LT: LT_val,
             SR_e: SR_e_val,
             C_S: C_S_val,
             C_C: C_C_val,
             DM: DM_val,
             CP: CP_val,
             CF: CF_val,
             AP: AP_val,
             W_Pn: W_Pn_val,
             SW_n: SW_n_val,
             S_w: S_w_val,
             S_el: S_el_val,
             D_i: D_i_val,
             E_co2: E_co2_val, 
             CT_el: CT_el_val,
             CT_n: CT_n_val,
             W_en: W_en_val,
             OF: OF_val,
             DW: DW_val}

# Groups of environmental categories

In [ ]:
# Group2 uses Diesel consumption


group1_05 = ["carcinogenic effects", "non-carcinogenic effects", "respiratory effects, inorganics",  "freshwater ecotoxicity",
             "freshwater eutrophication", "minerals and metals", "dissipated water", "land use"]  

group2_05 = ["climate change total", "ionising radiation", "ozone layer depletion", "photochemical ozone creation", \
            "freshwater and terrestrial acidification", "marine eutrophication", "terrestrial eutrophication", "fossils"]


group1_10 = ["climate change total",  "carcinogenic effects", "non-carcinogenic effects",\
              "respiratory effects, inorganics",  "freshwater ecotoxicity", "freshwater eutrophication", \
              "minerals and metals", "dissipated water", "fossils", "land use"]            

group2_10 = ["ionising radiation", "ozone layer depletion", "photochemical ozone creation", \
            "freshwater and terrestrial acidification", "marine eutrophication", "terrestrial eutrophication"]


group1_15 = ["climate change total", "carcinogenic effects", "ionising radiation", "non-carcinogenic effects", \
            "ozone layer depletion", "photochemical ozone creation", "respiratory effects, inorganics", \
            "freshwater and terrestrial acidification", "freshwater ecotoxicity", "freshwater eutrophication", \
            "minerals and metals", "dissipated water", "fossils", "land use"]

group2_15 = ["marine eutrophication", "terrestrial eutrophication"]

group_20 = [method[2] for method in ILCD]


# Simplified equations

In [ ]:
chi_1, chi_2, chi_3, chi_4, chi_5, chi_6, chi_7, delta_1, delta_2, delta_3, delta_4, delta_5, delta_6,\
delta_7, delta_8, delta_9 = \
symbols('chi_1, chi_2, chi_3, chi_4, chi_5, chi_6, chi_7, delta_1, delta_2, delta_3, delta_4, delta_5, \
delta_6, delta_7, delta_8, delta_9')


## Threshold 20% - one group only

In [ ]:
eq_group_20 = chi_1/(chi_2*P_ne - chi_3) + chi_4#
eq_group_20

## Threhold 15/10% - group 1

In [ ]:
eq_group1_15_10 = chi_1/(chi_2*P_ne - chi_3) + chi_4
eq_group1_15_10

## Threhold 15/10% - group 2

In [ ]:
eq_group2_15_10 = (D*delta_1 + delta_2) / (P_ne*delta_3 - delta_4) + delta_5
eq_group2_15_10

## Threshold 5% - group1

In [ ]:
eq_group1_05 = (SR_p*W_d*chi_1 + SR_p*chi_2 + W_d*chi_3 + chi_4) / (P_ne*SR_p*chi_5 - SR_p*chi_6) +chi_7
eq_group1_05

## Threshold 5% - group2

In [ ]:
eq_group2_05 = (D*SR_p*W_d*delta_1 + D*W_d*delta_2 + SR_p*W_d*delta_3 + SR_p*delta_4 + W_d*delta_5 + delta_6 ) \
/ (P_ne*SR_p*delta_7 - SR_p*delta_8) + delta_9
eq_group2_05

# Compute chi

## Threshold = 20%

In [ ]:
# Replace parameters in equation
eq_chi_20 = eq.subs(par_dict)
repl = {SR_p: SR_p_val,
        D: D_val,
        W_d: W_d_val}
eq_chi_20 = eq_chi_20.subs(repl)

chi_dict_20 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group_20:
        eq_chi_20_is = eq_chi_20.subs(is_dict)
        eq_chi_20_is = ratsimp(eq_chi_20_is)
              
        chi4 = eq_chi_20_is.args[0] 
        chi1 = fraction(eq_chi_20_is.args[1])[0]
        
        den = fraction(eq_chi_20_is.args[1])[1]
        chi2 = collect(den, P_ne, evaluate=False)[P_ne]
        chi3 = collect(den, P_ne, evaluate=False)[1]
        
        chi_dict_20[str(method)] = {'chi1': chi1, 'chi2': chi2,
                                   'chi3': chi3, 'chi4': chi4}
           

In [ ]:
# Show example of the formula
eq_chi_20_is

In [ ]:
chi_df_20 = pd.DataFrame.from_dict(chi_dict_20, orient='index')
chi_df_20 = chi_df_20.astype(float)
chi_df_20

## Threshold 15%

In [ ]:
# Select from chi_df_20 the categories that are in group1_15
mask =[chi_df_20.index.str.contains(method) for method in group1_15]
chi_df_15 = pd.concat([chi_df_20[m] for m in mask])
chi_df_15

## Threshold 10%

In [ ]:
# Select from chi_df_20 the categories that are in group1_15
mask =[chi_df_20.index.str.contains(method) for method in group1_10]
chi_df_10 = pd.concat([chi_df_20[m] for m in mask])
chi_df_10

## Threshold 5%

In [ ]:
# Replace parameters in equation
eq_chi_05 = eq.subs(par_dict)
repl = {D: D_val}
eq_chi_05 = eq_chi_05.subs(repl)

chi_dict_05 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group1_05:
        eq_chi_05_is = eq_chi_05.subs(is_dict)
        eq_chi_05_is = ratsimp(eq_chi_05_is)
              
        chi7 = eq_chi_05_is.args[0] 
        num =fraction(eq_chi_05_is.args[1])[0]
        chi1 = collect(num, [SR_p*W_d, SR_p, W_d], evaluate=False)[SR_p*W_d]
        chi2 = collect(num, [SR_p*W_d, SR_p, W_d], evaluate=False)[SR_p]
        chi3 = collect(num, [SR_p*W_d, SR_p, W_d], evaluate=False)[W_d]
        chi4 = collect(num, [SR_p*W_d, SR_p, W_d], evaluate=False)[1]
        
        den = fraction(eq_chi_05_is.args[1])[1]
        chi5 = collect(den, [P_ne*SR_p, SR_p], evaluate=False)[P_ne*SR_p]
        chi6 = collect(den, [P_ne*SR_p, SR_p], evaluate=False)[SR_p]
        
        chi_dict_05[str(method)] = {'chi1': chi1, 'chi2': chi2,
                                    'chi3': chi3, 'chi4': chi4,
                                    'chi5': chi5, 'chi6': chi6,
                                    'chi7': chi7}
           

In [ ]:
# Show example of the formula
eq_chi_05_is

In [ ]:
chi_df_05 = pd.DataFrame.from_dict(chi_dict_05, orient='index')
chi_df_05 = chi_df_05.astype(float)
chi_df_05

# Compute delta

## Threshold = 15%

In [ ]:
# Replace parameters in equation
eq_delta_15 = eq.subs(par_dict)
repl = {SR_p: SR_p_val,
        W_d: W_d_val}
eq_delta_15 = eq_delta_15.subs(repl)

delta_dict_15 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group2_15:
        eq_delta_15_is = eq_delta_15.subs(is_dict)
        eq_delta_15_is = ratsimp(eq_delta_15_is)
              
        delta5 = eq_delta_15_is.args[0] 
        
        num  = fraction(eq_delta_15_is.args[1])[0]
        delta1 = collect(num, [D], evaluate=False)[D]
        delta2 = collect(num, [D], evaluate=False)[1]
        
        den  = fraction(eq_delta_15_is.args[1])[1]
        delta3 = collect(den, [P_ne], evaluate=False)[P_ne]
        delta4 = collect(den, [P_ne], evaluate=False)[1]
                
        delta_dict_15[str(method)] = {'delta1': delta1, 'delta2': delta2,
                                    'delta3': delta3, 'delta4': delta4,
                                    'delta5': delta5}

In [ ]:
# Show example of the formula
eq_delta_15_is

In [ ]:
delta_df_15 = pd.DataFrame.from_dict(delta_dict_15, orient='index')
delta_df_15 = delta_df_15.astype(float)
delta_df_15

## Threshold = 10%

In [ ]:
# Replace parameters in equation
eq_delta_10 = eq.subs(par_dict)
repl = {SR_p: SR_p_val,
        W_d: W_d_val}
eq_delta_10 = eq_delta_10.subs(repl)

delta_dict_10 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group2_10:
        eq_delta_10_is = eq_delta_10.subs(is_dict)
        eq_delta_10_is = ratsimp(eq_delta_10_is)
              
        delta5 = eq_delta_10_is.args[0] 
        
        num  = fraction(eq_delta_10_is.args[1])[0]
        delta1 = collect(num, [D], evaluate=False)[D]
        delta2 = collect(num, [D], evaluate=False)[1]
        
        den  = fraction(eq_delta_10_is.args[1])[1]
        delta3 = collect(den, [P_ne], evaluate=False)[P_ne]
        delta4 = collect(den, [P_ne], evaluate=False)[1]
                
        delta_dict_10[str(method)] = {'delta1': delta1, 'delta2': delta2,
                                    'delta3': delta3, 'delta4': delta4,
                                    'delta5': delta5}

In [ ]:
# Show example of the formula
eq_delta_15_is

In [ ]:
delta_df_10 = pd.DataFrame.from_dict(delta_dict_10, orient='index')
delta_df_10 = delta_df_10.astype(float)
delta_df_10

## Threshold = 5%

In [ ]:
# Replace parameters in equation
eq_delta_05 = eq.subs(par_dict)

delta_dict_05 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group1_05:
        eq_delta_05_is = eq_delta_05.subs(is_dict)
        eq_delta_05_is = ratsimp(eq_delta_05_is)
              
        delta9 = eq_delta_05_is.args[0] 
        
        num = fraction(eq_delta_05_is.args[1])[0]
        delta1 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[D*SR_p*W_d]
        delta2 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[D*W_d]
        delta3 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[SR_p*W_d]
        delta3 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[SR_p]
        delta5 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[W_d]
        delta6 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[1]
        
        den = fraction(eq_delta_05_is.args[1])[1]
        delta7 = collect(den, [P_ne*SR_p, SR_p], evaluate=False)[P_ne*SR_p]
        delta8 = collect(den, [P_ne*SR_p, SR_p], evaluate=False)[SR_p]
        
        delta_dict_05[str(method)] = {'delta1': delta1, 'delta2': delta2,
                                    'delta3': delta3, 'delta4': delta4,
                                    'delta5': delta5, 'delta6': delta6,
                                    'delta7': delta7, 'delta8': delta8,
                                    'delta9': delta9}

In [ ]:
# Show example of the formula
eq_delta_05_is

In [ ]:
delta_df_05 = pd.DataFrame.from_dict(delta_dict_05, orient='index')
delta_df_05 = delta_df_05.astype(float)
delta_df_05

# Save to excel

In [ ]:
with pd.ExcelWriter(os.path.join(path, 'generated_files\Simplified models coefficients enhanced - symbolic - thresholds.xlsx')) as writer:
    chi_df_05.to_excel(writer, sheet_name='chi_5%')
    chi_df_10.to_excel(writer, sheet_name='chi_10%')
    chi_df_15.to_excel(writer, sheet_name='chi_15%')
    chi_df_20.to_excel(writer, sheet_name='chi_20%')    
    delta_df_05.to_excel(writer, sheet_name='delta_5%')
    delta_df_10.to_excel(writer, sheet_name='delta_10%')
    delta_df_15.to_excel(writer, sheet_name='delta_15%')
    